<h1>Brain Tumor dataset generator from images</h1>

<h3>Cette partie va permettre de tenter d'obtenir la segmentation des tumeurs.

Segmentation sera obtenue avec Unet 

Unet https://pytorch.org/hub/mateuszbuda_brain-segmentation-pytorch_unet/

Features seront obtenus avec la classe radiomics.shape2D.RadiomicsShape2D

https://pyradiomics.readthedocs.io/en/latest/features.html#module-radiomics.shape2D<

Installation des dépendances pour pyradiomics

In [1]:
!pip install pyradiomics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 28.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyradiomics: filename=pyradiomics-3.0.1-cp310-cp310-linux_x86_64.whl size=117059 sha256=51456a26b2b380ab6851ed52e37c9aca743d9737b14690ab1dac2b0cb4cbd33e
  Stored in directory: /root/.cache/pip/wheels/91/c5/13/c5fd4c5ad3edf4062bb3855bd66fad25871c9c6dc0b3fda544
Successfully built pyradiomics


In [2]:
import os
import torch # use to load mateuszbuda/brain-segmentation-pytorch pre-entrained model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pydicom import dcmread # image dicom reader
import SimpleITK as sitk # image filtering
from PIL import Image #  PIL is an Python library used for image processing
import matplotlib.pyplot as plt
import radiomics

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<h2>Init</h2>


Load mateuszbuda/brain-segmentation-pytorch, U-Net with batch normalization for biomedical image segmentation with pretrained weights for abnormality segmentation in brain MRI

In [3]:
segmentation_model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=True, trust_repo=False)

The repository mateuszbuda_brain-segmentation-pytorch does not belong to the list of trusted repositories and as such cannot be downloaded. Do you trust this repository and wish to add it to the trusted list of repositories (y/N)? y


Downloading: "https://github.com/mateuszbuda/brain-segmentation-pytorch/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/mateuszbuda/brain-segmentation-pytorch/releases/download/v1.0/unet-e012d006.pt" to /root/.cache/torch/hub/checkpoints/unet-e012d006.pt


In [4]:
root_path = "../input/rsna-miccai-brain-tumor-radiogenomic-classification"
img_path = root_path+"/train"
label_file = root_path+'/train_labels.csv'

In [5]:
# définit le "reader"
# Read serie of image files into a SimpleTK image
sitk_reader = sitk.ImageSeriesReader()
sitk_reader.LoadPrivateTagsOn()

<h3>definition de la fonction de récupération des images de chaque patient et regroupement pour segmentation</h3>

In [6]:
def get_processed_image(BraTS21ID):
    '''
    definition de la fonction de récupération des images de chaque patient et regroupement pour segmentation
    in : BraTS21ID id du patient
    out : image constituée des différentes images du patient
    '''
    # LE MODELE DE SEGMENTATION LIMITE A 3 COUCHES
    # IMPASSE SUR T2W
    
    # chemin des sequences d'images
    t1w_path = f'{img_path}/{str(BraTS21ID).zfill(5)}/T1w'
    flair_path = f'{img_path}/{str(BraTS21ID).zfill(5)}/FLAIR'
    t1wce_path = f'{img_path}/{str(BraTS21ID).zfill(5)}/T1wCE'
    #t2w_path = f'{img_path}/{str(BraTS21ID).zfill(5)}/T2w'

    # GetGDCMSeriesFileNames : 
    # ----------------------
    #Generate a sequence of filenames from a directory with a DICOM data set and a series ID. 
    #This method generates a sequence of filenames whose filenames point to DICOM files. 
    #The data set may contain multiple series. 
    
    t1w_image = sequence_filenames(t1w_path)
    flair_image = sequence_filenames(flair_path)
    t1wce_image = sequence_filenames(t1wce_path)
    #t2w_image = sequence_filenames(t2w_path)
            
    # re-sampling
    re_sampled_flair = re_sample_image(flair_image, t1w_image)
    re_sampled_t1wce = re_sample_image(t1wce_image, t1w_image)
    #re_sampled_t2w = re_sample_image(t2w_image, t1w_image)

    # normalisation
    t1w_array = normalize(sitk.GetArrayFromImage(t1w_image))
    flair_array = normalize(sitk.GetArrayFromImage(re_sampled_flair))
    t1wce_array = normalize(sitk.GetArrayFromImage(re_sampled_t1wce))
    #t2w_array = normalize(sitk.GetArrayFromImage(re_sampled_t2w))

    sequence_stacked = np.stack([t1w_array, flair_array, t1wce_array]) #, t2w_array])

    central_slice = t1w_array.shape[0] // 2
    rvb = sequence_stacked[:, central_slice, :, :].transpose(1, 2, 0)
    image = Image.fromarray((rvb * 255).astype(np.uint8))
    return  np.array([np.moveaxis(np.array(image.resize((256, 256))), -1, 0)])

In [7]:
def sequence_filenames(path) :
    '''
    # GetGDCMSeriesFileNames : 
    # ----------------------
    #Generate a sequence of filenames from a directory with a DICOM data set and a series ID. 
    #This method generates a sequence of filenames whose filenames point to DICOM files. 
    #The data set may contain multiple series. 
    in : path le chemin des fichiers
    out : la série
    '''
    filenames = sitk_reader.GetGDCMSeriesFileNames(path)
    sitk_reader.SetFileNames(filenames)
    image = sitk_reader.Execute()
    
    return image    

In [8]:
def normalize(dataset) :
    return (dataset - np.min(dataset)) / (np.max(dataset) - np.min(dataset))

In [9]:
def re_sample_image(image, ref_img):
    '''
    definition de la fonction de récupération des images de chaque patient et regroupement pour segmentation
    in : BraTS21ID id du patient
    out : image constituée des différentes images du patient
    '''
    re_sampler = sitk.ResampleImageFilter()
    
    re_sampler.SetReferenceImage(ref_img)

    re_sampler.SetDefaultPixelValue(image.GetPixelIDValue())
    re_sampler.SetInterpolator(sitk.sitkLinear)
    re_sampler.SetOutputSpacing(ref_img.GetSpacing())
    re_sampler.SetOutputDirection(ref_img.GetDirection())
    re_sampler.SetOutputOrigin(ref_img.GetOrigin())
    re_sampler.SetSize(ref_img.GetSize())
    re_sampler.SetTransform(sitk.AffineTransform(image.GetDimension()))

    re_sampled_image = re_sampler.Execute(image)
    
    return re_sampled_image

In [10]:
def segmentation_process(image_resized):
    '''
    in : re_sampled_image
    out : segmented image
    '''
    #img_src = np.array([np.moveaxis(np.array(re_sampled_image.resize((256, 256))), -1, 0)])
    segmentation = segmentation_model(torch.Tensor(image_resized))
    return segmentation

In [11]:
def show_segmentation(img_src,segmentation) :
    '''
    in : img_src is re_sampled_image resized, segmentation is the result of segmentation_model application
    '''
    titles=['T1w','FLAIR','T1wce'] #,'T2w']
    for i in range(3):
        plt.subplot(1,4,1+i)
        plt.imshow(img_src[0, i])
        plt.title(titles[i])
        plt.xticks([])
        plt.yticks([])
    
    plt.subplot(1,4,4)
    plt.imshow(segmentation.detach().numpy()[0,0])
    plt.title('Segmentation')
    plt.xticks([])
    plt.yticks([])

    plt.tight_layout()
    plt.show()

Structure des DF

In [18]:
# DataFrame
def init_dataset_radiomics() :
    '''
    Création à vide des dataset d'acquisition des données Radiomics
    '''
    global df_shapes
    global df_textures
    global df_first_orders_features
    
    df_shapes_columns = ['ID','BraTS21ID','MeshSurface','PixelSurface','Perimeter','PerimeterSurfaceRatio','Sphericity',
                              'SphericalDisproportion','MaximumDiameter','MajorAxisLength','MinorAxisLenth','Elongation']
    df_shapes = pd.DataFrame(columns=df_shapes_columns) 


    df_textures_columns = ['ID','Autocorrelation', 'ClusterProminence', 'ClusterShade', 'ClusterTendency', 'Contrast', 
                           'Correlation', 'DifferenceAverage', 'DifferenceEntropy', 'DifferenceVariance', 'Id', 'Idm', 
                           'Idmn', 'Idn', 'Imc1', 'Imc2', 'InverseVariance', 'JointAverage', 'JointEnergy', 'JointEntropy', 
                           'MCC', 'MaximumProbability', 'SumAverage', 'SumEntropy', 'SumSquares']
    df_textures = pd.DataFrame(columns=df_textures_columns) 


    df_first_orders_features_columns=['ID','10Percentile', '90Percentile', 'Energy', 'Entropy', 'InterquartileRange', 'Kurtosis', 
                                      'Maximum', 'MeanAbsoluteDeviation', 'Mean', 'Median', 'Minimum', 'Range', 'RobustMeanAbsoluteDeviation', 
                                      'RootMeanSquared', 'Skewness', 'TotalEnergy', 'Uniformity', 'Variance']
    df_first_orders_features = pd.DataFrame(columns=df_first_orders_features_columns) 

In [13]:
def add_patient_data(ID,img_resized,segmentation) :
    '''
    Ajoute des données tirées des images pour le dataset d'analyse
    in : ID (patient), img_resized,segmentation
    '''
    global df_shapes
    global df_textures
    global df_first_orders_features
    
    # shape
    results = radiomics.shape2D.RadiomicsShape2D(
        sitk.GetImageFromArray(img_resized), 
        sitk.GetImageFromArray(np.array([
            segmentation[0][0].detach().cpu().numpy() > 0.5
        ]).astype(np.uint8)),
        force2D=True
    )
    
    shape2D = {}
    shape2D['ID'] = int(ID)
    shape2D['BraTS21ID'] = int(ID)
    shape2D['MeshSurface'] = results.getMeshSurfaceFeatureValue()
    shape2D['PixelSurface'] = results.getPixelSurfaceFeatureValue()
    shape2D['Perimeter'] = results.getPerimeterFeatureValue()
    shape2D['PerimeterSurfaceRatio'] = results.getPerimeterSurfaceRatioFeatureValue()
    shape2D['Sphericity'] = results.getSphericityFeatureValue()
    shape2D['SphericalDisproportion'] = results.getSphericalDisproportionFeatureValue()
    shape2D['MaximumDiameter'] = results.getMaximumDiameterFeatureValue()
    shape2D['MajorAxisLength'] = results.getMajorAxisLengthFeatureValue()
    shape2D['MinorAxisLenth'] = results.getMinorAxisLengthFeatureValue()
    shape2D['Elongation'] = results.getElongationFeatureValue()
    
    df_shapes=df_shapes.append(shape2D,ignore_index=True)
    
    # GLCM
    results=radiomics.glcm.RadiomicsGLCM(
        sitk.GetImageFromArray(img_resized[0,0,:,:].reshape(1, 256, 256)), 
        sitk.GetImageFromArray(np.array([
            segmentation[0][0].detach().cpu().numpy() > 0.5
        ]).astype(np.uint8)),
        force2D=True
    )

    results.enableAllFeatures()
    res = results.execute()
    res['ID']=int(ID)
    #print(res)
    df_textures=df_textures.append(res,ignore_index=True)
    
    # First-orders features
    results =  radiomics.firstorder.RadiomicsFirstOrder(
        sitk.GetImageFromArray(img_resized[0,0,:,:].reshape(1, 256, 256)), 
        sitk.GetImageFromArray(np.array([
            segmentation[0][0].detach().cpu().numpy() > 0.5
        ]).astype(np.uint8)),
        force2D=True
    )

    results.enableAllFeatures()
    res = results.execute()
    res['ID']=int(ID)
    #print(res)
    df_first_orders_features=df_first_orders_features.append(res,ignore_index=True)

<h3>Extraction des données de ces images pour le patient 0</h3>

In [14]:
# test sur patient 0
'''
init_dataset_radiomics()
img_resized = get_processed_image(0)
segmentation = segmentation_process(img_resized)
show_segmentation(img_resized,segmentation)
add_patient_data(0,img_resized,segmentation)

print(df_shapes.head()) 
print(df_textures.head())
print(df_first_orders_features.head())
'''

'\ninit_dataset_radiomics()\nimg_resized = get_processed_image(0)\nsegmentation = segmentation_process(img_resized)\nshow_segmentation(img_resized,segmentation)\nadd_patient_data(0,img_resized,segmentation)\n\nprint(df_shapes.head()) \nprint(df_textures.head())\nprint(df_first_orders_features.head())\n'

<h2>Création du dataset</h2>

In [19]:
# Création à vide des datasets
init_dataset_radiomics()
# Charge le fichier des résultats avec les ID patients (jeu d'entrainement qui va nous servir sur l'ensemble)
labels = pd.read_csv(label_file) 

array_img_resized = []
array_segmentation = []

for i in labels.BraTS21ID : #.head():
    print(i)
    img_resized = get_processed_image(i)
    segmentation = segmentation_process(img_resized)
    
    #array_img_resized.append(img_resized)
    #array_segmentation.append(segmentation)
    
    add_patient_data(i,img_resized,segmentation)

    
#for i in range(5):
#    show_segmentation(array_img_resized[i],array_segmentation[i])

# jointure des 3 datasets
df_shapes=df_shapes.set_index('ID')
df_textures=df_textures.set_index('ID')
df_first_orders_features=df_first_orders_features.set_index('ID')

#print(df_shapes.head()) 
#print(df_textures.head())
#print(df_first_orders_features.head())

df = df_shapes
df = df.join(df_textures)
df = df.join(df_first_orders_features)

df['BraTS21ID']=df['BraTS21ID'].astype(int)
print(df.dtypes)
print(df.head())
df.to_csv('brain_tumor_dataset.csv', index = True) 
print("Terminé")

0


ImageSeriesReader (0x587d41f25150): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000398747

/tmp/ipykernel_35/1660570955.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_shapes=df_shapes.append(shape2D,ignore_index=True)
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
/tmp/ipykernel_35/1660570955.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_textures=df_textures.append(res,ignore_index=True)
/tmp/ipykernel_35/1660570955.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_first_orders_features=df_first_orders_features.append(res,ignore_index=True)


2


/tmp/ipykernel_35/1660570955.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_shapes=df_shapes.append(shape2D,ignore_index=True)
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
/tmp/ipykernel_35/1660570955.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_textures=df_textures.append(res,ignore_index=True)
/tmp/ipykernel_35/1660570955.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_first_orders_features=df_first_orders_features.append(res,ignore_index=True)


3


...

1009


/tmp/ipykernel_35/1660570955.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_shapes=df_shapes.append(shape2D,ignore_index=True)
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
/tmp/ipykernel_35/1660570955.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_textures=df_textures.append(res,ignore_index=True)
/tmp/ipykernel_35/1660570955.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_first_orders_features=df_first_orders_features.append(res,ignore_index=True)


1010


ImageSeriesReader (0x587d444c8680): Non uniform sampling or missing slices detected,  maximum nonuniformity:1.97312

/tmp/ipykernel_35/1660570955.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_shapes=df_shapes.append(shape2D,ignore_index=True)
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
/tmp/ipykernel_35/1660570955.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_textures=df_textures.append(res,ignore_index=True)
/tmp/ipykernel_35/1660570955.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_first_orders_features=df_first_orders_features.append(res,ignore_index=True)


BraTS21ID                        int64
MeshSurface                    float64
PixelSurface                   float64
Perimeter                      float64
PerimeterSurfaceRatio          float64
Sphericity                     float64
SphericalDisproportion         float64
MaximumDiameter                float64
MajorAxisLength                float64
MinorAxisLenth                 float64
Elongation                     float64
Autocorrelation                 object
ClusterProminence               object
ClusterShade                    object
ClusterTendency                 object
Contrast                        object
Correlation                     object
DifferenceAverage               object
DifferenceEntropy               object
DifferenceVariance              object
Id                              object
Idm                             object
Idmn                            object
Idn                             object
Imc1                            object
Imc2                     

Donnés de "Texture"

first-order features

<h3>Sépare jeu d'entrainement et d'essai<h3>

In [ ]:
targets = labels[:5]
targets=targets.set_index('BraTS21ID')

df = pd.read_csv('brain_tumor_dataset.csv')
df=df.set_index('BraTS21ID')

X_train, X_test, y_train, y_test = train_test_split(df.drop('ID',axis=1), targets.MGMT_value, test_size=0.2, random_state=42)

# X_train and X_test will be completed later

print(X_train.head())
print(y_train.head())
# L'accès au dossier patient ce sera en formattant ld BraTS21ID
# str(labels.iloc[0,0]).zfill(5) par exemple pour le patient identifié 0

In [ ]:
'''
data_path = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/"

y_train = pd.read_csv(data_path+'train_labels.csv')
y_test = pd.read_csv(data_path+'sample_submission.csv')

print(y_train.head(10))
print(y_test.head(10))

y_train.info()
y_test.info()
'''


In [ ]:
'''from pydicom import dcmread
import pydicom
import matplotlib.pyplot as plt
print(pydicom.__version__)

#os.chdir("/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T2w")
#print (os.getcwd())
from pydicom import dcmread
x=dcmread("/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/Image-106.dcm")
print(x.dir())
# infos patient
print(x.dir("pat"))
print(x.pixel_array)
plt.imshow(x.pixel_array)'''